In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
prompt = ChatPromptTemplate.from_template(
    "用三个emoji描述：{item}"
)
model = ChatOpenAI(openai_api_key=api_key)
chain = prompt | model

response = chain.invoke({"item": "程序员的工作日常"})
print(response.content)


💻📚🔍


### from langchain_core.prompts import PipelinePromptTemplate

官方文档显示将会逐步弃用PipelinePromptTemplate
https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.pipeline.PipelinePromptTemplate.html

取而代之的是将代码中的单个提示链接在一起。例如，使用for循环执行操作

In [2]:
from langchain.prompts import PromptTemplate

Character_template = "你是{person}, 你有着{attribute}\n"
Character_prompt = PromptTemplate.from_template(Character_template)
behavior_template = "你会遵从以下行为：\n{behavior_list}\n"
behavior_prompt = PromptTemplate.from_template(behavior_template)
prohibit_template = "你不能遵从以下行为：\n{prohibit_list}\n"
prohibit_prompt = PromptTemplate.from_template(prohibit_template)
pipeline_prompts = [("Character", Character_prompt),("Behavior", behavior_prompt),("Prohibit", prohibit_prompt),]

final_template = """{Character}{Behavior}{Prohibit}"""
final_prompt = PromptTemplate.from_template(final_template)

my_input = {"person": "科学家", "attribute": "严谨", "behavior_list": "1. 确保实验结果准确\n2. 记录实验过程\n3. 分析实验数据", "prohibit_list": "1. 不做实验\n2. 不记录实验过程\n3. 不分析实验数据"}
for name, prompt in pipeline_prompts:
    my_input[name] = prompt.invoke(my_input).to_string()
my_output = final_prompt.invoke(my_input).to_string()
print(my_output)


你是科学家, 你有着严谨
你会遵从以下行为：
1. 确保实验结果准确
2. 记录实验过程
3. 分析实验数据
你不能遵从以下行为：
1. 不做实验
2. 不记录实验过程
3. 不分析实验数据



In [3]:
# 序列化模板使用
from langchain_core.prompts import load_prompt

# yaml文件
prompt = load_prompt("./test_load/prompt.yaml", encoding="utf-8")
print(prompt.format(name="科学家", what="热爱搞发明"))

# json文件
prompt = load_prompt("./test_load/prompt.json", encoding="utf-8")
print(prompt.format(name="科学家", what="热爱搞发明"))

给我讲一个关于科学家的热爱搞发明的故事
给我讲一个关于科学家的热爱搞发明的故事


### 样例选择器：自定义、长度选样、MMR样例选择器、n-gram重叠度样例、相似度样例选择器

In [4]:
# 相似度样例选择器 使用OpenAIEmbeddings

from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]


In [5]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples, 
    # OpenAIEmbeddings: embedding class 用于生成用于测量语义相似性的embedding嵌入向量
    OpenAIEmbeddings(), 
    # Chroma: a VectorStore class 用于存储嵌入信息并进行相似性搜索
    Chroma, 
    # k: 需要的相似性样本量
    k=1
)
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",  # 先输出句首  再输出选择到的相似性的例子
    suffix="Input: {adjective}\nOutput:",      # 最后输出句尾
    input_variables=["adjective"],
)
print(similar_prompt.format(adjective="worried"))

Give the antonym of every input

Input: happy
Output: sad

Input: worried
Output:


### stream流式输出

In [6]:
from langchain_ollama import OllamaLLM
llm = OllamaLLM(
    model="deepseek-r1:1.5b",
    temperature=0.7,
    base_url="http://localhost:11434"
)
for chunk in llm.stream("胸有成竹是什么意思？一个简短的解释"):
    print(chunk, end="", flush=True)

<think>
嗯，我现在想了解“胸有成竹”到底是什么意思。首先，“胸”指的是心脏，而“成竹”就是得到了成果。所以这句话的意思应该是说，胸中有自己，才会有好的结果。

可是，我不太确定这个说法是不是正确或者常见。我记得以前学过一些成语和谚语，可能和这个有关。比如说，“胸有成竹”这个词在中文里确实是一个常用的表达，意思是说一个人有了自己的心，才能取得成果。

那这句话的结构是怎样的呢？“胸”是主词，“有成竹”是宾语。这句话的意思应该是，拥有自己的心脏（即独立、自立），就能成就事业或目标。

我还记得有没有其他类似的成语或者俗语用在类似的位置上。比如“有志于行者无难”，这个意思是对人的态度很有要求，但可能不是直接对应的。另一个例子是“胸有成竹”和“穷则变，达则恒”这样的表达，前者强调独立，后者讲适应环境后才能稳定发展。

所以，“胸有成竹”主要是形容一个人有了自己的心，能够自己去努力，最终取得成功。这可能包括个人的独立、自我觉察以及积极面对挑战的能力。

有没有例外的情况呢？比如说，在某些情况下，虽然没有完全自立，但也能取得成果。比如，家庭成员共同支持，或者社区里的帮助，虽然没有完全独立，但也有可能取得成就。所以，“胸有成竹”更多是强调独立和自信的重要性。

我是不是漏掉了什么例子？比如在教育或事业中，很多成功人士都是通过自己的努力和坚持获得的成果，而不是依赖他人或急于求成。所以，“胸有成竹”确实反映了这种独立自主、自立自强的精神。

总结一下，“胸有成竹”意思是拥有自己的心，能够通过自己的努力取得结果，强调了独立自主的重要性。
</think>

“胸有成竹”是一个汉语成语，表达个人独立、自信并能自我实现的意思。其核心含义是指拥有自己的独立性和自立意识，能够在自己有能力的情况下取得成果或成就。这不仅体现在个人的事业和个人生活上，还可能延伸到社会和社区层面，强调通过努力和坚持能够实现目标。

In [7]:
# flush=True 确保每个片段立即显示，而不是等待缓冲区填满。
# flush=False 有时候可能会有部分字符被吞，比如:

In [8]:
for chunk in llm.stream("胸有成竹是什么意思？一个简短的解释"):
    print(chunk, end="", flush=False)

<think>
需要理解“胸有成竹”这个词的意思。首先，“胸”在这里指的是人的躯干，是身体的一部分。“成竹”则是指完成了某件成就或目标。

胸有成竹”应该是指一个人在 physically 和 mentally 上都有足够的准备和能力去做到某件事或者达成某个目标。换句话说，一个人如果能有“胸中有成竹”，那他的心理和身体条件都很成熟，能够应对各种挑战和任务。

让我想到，不仅仅是个人努力，还有外部环境的支持也起作用。比如，学习一门新技能可能需要时间和努力，但如果有足够的心灵准备，加上环境的鼓励，就可以完成目标。

记得以前看过一些名言或者故事，看到那些成功人士如何通过调整心态、培养内在能力来克服困难。他们可能并没有完全准备好，但他们通过自己的努力和选择，最终达到了他们的目标。

，“胸有成竹”不仅指身体上的准备，还包括心理上的平衡和支持。这意味着在面对任何挑战时，一个人都能调整心态，找到适合自己的方法，从而成功解决问题或达成目标。
</think>

成竹”是指一个人在 physically 和 mentally 上都有足够的准备和能力去做到某件事或者达成某个目标。具体来说，这个词语强调了一个人在内在和外在条件上的成熟程度，能够应对各种挑战和任务。

，“胸有成竹”意味着一个人不仅身体上准备好承担责任，心理上也足够平衡和支持，能够在面对困难时保持积极的心态，并通过自己的努力实现目标。

In [9]:
# 也可以用聊天模型，主要是使用 model.stream()

from langchain_ollama import ChatOllama
model = ChatOllama(
    model="deepseek-r1:1.5b",
    temperature=0.7,
    base_url="http://localhost:11434"
)
messages = [
    ("human", "胸有成竹是什么意思？一个简短的解释"),
]
for chunk in model.stream(messages):
    print(chunk.content, end='', flush=True)

<think>
好的，我现在要理解“胸有成竹”这个词的意思。首先，“胸”指的是胸口，也就是人的肩膀和头部的部分，通常象征着坚强、勇敢和力量。而“成竹”是指得到了成功或成就。

所以，把这两个词结合起来，“胸有成竹”应该意思是拥有坚实的基础，能够支撑住事情的发展，最终达成目标。这可能与一个人的成长和发展有关，表现为从失败到成功的转变，或者在面对挑战时保持积极的心态。

再想想，这个表达常用于鼓励人去接受自己的困难和挑战，相信自己有能力克服。它强调了内在的力量和外在的努力的结合，而不是单纯依靠外部环境的帮助。

比如，一个人如果长期处于低谷或失败中，可能会感到迷茫和无助，这时候“胸有成竹”就能帮助他们找到新的方向，重新振作起来，最终实现自己的目标。

总结一下，“胸有成竹”意思是拥有坚实的基础和坚定的信念，能够克服困难并达成目标。这可能与内在的成长、外在的努力以及积极的心态紧密相关。
</think>

“胸有成竹”是指一个人在面对困难和挑战时，拥有坚实的心理基础和支持力，能够在成功的过程中不断成长和发展。这句话强调了内在力量和外在努力的结合，鼓励人们以积极的心态去面对问题，并相信自己能够克服困难并实现目标。

**解释：**

1. **胸有成竹的意义：**
   - 胸意味着人的肩膀和头部，象征着坚强、勇敢和力量。
   - 成竹指成功或成就。
   - “胸有成竹”意为拥有坚实的基础和支撑能力，能够应对挑战并达成目标。

2. **核心含义：**
   - 代表内在的力量与外在的努力结合，鼓励积极面对困难。
   - 强调自信和决心，不被外界压力所左右，而是相信自己可以成功。

3. **适用场景：**
   - 在个人成长或职业生涯中，用来激励人调整心态，避免退缩，保持持续努力。
   - 指使人们在困境中保持积极的心态，相信自己的潜力和能力。

4. **情感意义：**
   - 培养自信，增强自我信心，避免因外部因素影响而感到沮丧或自责。
   - 养成持续行动的习惯，推动个人或组织不断进步和发展。

总之，“胸有成竹”不仅是对成功的一种承诺，更是对自己力量和信念的肯定，鼓励人们在困难面前保持坚定，朝着目标前进。

### 统计token消耗

In [10]:
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage

llm = ChatOllama(
    model="deepseek-r1:1.5b",
    temperature=0.7,
    base_url="http://localhost:11434",
    stream_usage=True 
)

response = llm.invoke([
    HumanMessage(content="望梅止渴是什么意思")
])

# 输出内容和token统计
print(f"响应内容: {response.content}")
print(f"输入Token: {response.usage_metadata['input_tokens']}")
print(f"输出Token: {response.usage_metadata['output_tokens']}")
print(f"总消耗Token: {response.usage_metadata['total_tokens']}")

响应内容: <think>
好的，我现在要弄清楚“望梅止渴”是什么意思。首先，“望”这个词在中文里有几种解释，比如仰望、观看、观察，或者象征性的，比如欣赏。而“梅”是梅花的意思，通常用来形容美丽的梅花。“渴”则是表示渴求、渴望的意思。

那么，“望梅止渴”就是说把希望寄托于梅花上，希望能够停止渴求，也就是达到期待的境界。梅花在寒冷的季节盛开，能够象征着坚韧不拔的精神和强大的意志力。所以，望梅止渴可能是在表达一种积极向上的心态，相信通过努力，自己能够克服困难，达到自己的目标。

我还记得以前学过一个成语或者俗语，是“望梅止渴”，可能是指在遇到困难或挫折时，选择暂时放慢脚步，不急于解决问题，而是耐心等待，直到时机来临时，再采取行动。这和“望梅止渴”有相似的意思，都是通过等待或等待机会来实现目标。

另外，“望梅止渴”也常常用来形容某人对某些事情保持乐观的态度，即使面对困难，也能坚持不懈地去克服，最终取得成功。所以它不仅是一种心理上的状态，也是一种行为上的选择，让人保持积极的心态，相信自己能行。

我还想知道“望梅止渴”在文学作品中有没有出现过，或者是成语的一种变体。可能是在某些诗文中，作者通过描绘梅花的美丽来表达对未来的希望或坚持的精神。比如，写到梅花独自绽放，象征着坚韧不拔的人格特征，而通过望其止渴的态度，表达了对自己未来的坚定信念。

另外，“望梅止渴”这个词的用法是不是很广泛呢？可能在各种演讲、写作或者口语中都能听到，是一种简洁有力的表达方式，能够传达出积极向上的 message。它不仅仅是字面上的梅花，更像是一种心理上的引导，让人从内心深处去面对挑战和困难。

我还想确认一下，“望梅止渴”是不是一种成语或者俗语，还是一个比较常见的词语。查了一下字典和用法书，“望梅止渴”确实是一个成语，常用来比喻在追求某个目标时，由于环境或条件的原因，选择暂时放慢脚步，等待时机来临时，再行动。它表达了一种积极向上的态度。

此外，“望梅止渴”也常常被用来形容一个人对某些事情保持乐观的态度，即使面对困难和挑战，也能坚持不懈地去克服，最终取得成功。这种心态不仅体现在个人生活中，也可以在团队合作中体现出来，让人团结一心，共同面对问题。

我还想知道“望梅止渴”是否有其特定的语境或文化背景，它是否只适用于某些地区或者特定的历史时期。可能在古代汉语中，“望梅止渴”并不是特别常用，更多是在文

### 加载以下类型文件
.md .json .txt .csv .html .pdf

### .md类型文件

In [11]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./test_load/example.md")
docs = loader.load()
print(docs[0].page_content) 
print(docs[0].metadata)

# This is an example of the makedown format.
* Life is really simple, but people insist on making it complicated.
{'source': './test_load/example.md'}


### .json类型文件

In [12]:
from langchain_community.document_loaders import JSONLoader

loader = JSONLoader(
    file_path="./test_load/example.json",
    jq_schema=".city"
)
# jq_schema参数在 JSONLoader中使用的是 jq查询语言
# jq_schema=".city" 查询city
docs = loader.load()
print(docs[0].page_content)
print(docs[0].metadata)


New York
{'source': 'D:\\AI\\test_load\\example.json', 'seq_num': 1}


### .txt类型文件

In [13]:
loader = TextLoader("./test_load/example.txt")
docs = loader.load()
print(docs[0].page_content)  
print(docs[0].metadata)

This is an example of the txt format.
You are all on your own.
{'source': './test_load/example.txt'}


### .csv类型文件

In [14]:
loader = TextLoader("./test_load/example.csv")
docs = loader.load()
print(docs[0].page_content)  
print(docs[0].metadata)

Name,Age,City
Alice,25,New York
Bob,30,Los Angeles
Charlie,35,Chicago

{'source': './test_load/example.csv'}


### .html类型文件

In [15]:
from langchain_community.document_loaders import UnstructuredHTMLLoader
loader = UnstructuredHTMLLoader("./test_load/example.html")
docs = loader.load()
print(docs[0].page_content)  
print(docs[0].metadata)

我的第一个标题

我的第一个段落。
{'source': './test_load/example.html'}


### .pdf类型文件

In [16]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("./test_load/example.pdf")
docs = loader.load()
print(docs[0].page_content)  
print(docs[0].metadata)

Hello, this is a PDF file created with FPDF!
This is the second line of text.
{'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20250226165734', 'source': './test_load/example.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}


### Excel文件

In [17]:
from langchain.document_loaders import UnstructuredExcelLoader
loader = UnstructuredExcelLoader("./test_load/example.xlsx", mode="elements")
docs = loader.load()
print(docs[0].page_content)  
print(docs[0].metadata)

Name Age City Alice 25 New York Bob 30 Los Angeles Charlie 35 Chicago
{'source': './test_load/example.xlsx', 'file_directory': './test_load', 'filename': 'example.xlsx', 'last_modified': '2025-02-26T17:02:48', 'page_name': 'Sheet1', 'page_number': 1, 'text_as_html': '<table><tr><td>Name</td><td>Age</td><td>City</td></tr><tr><td>Alice</td><td>25</td><td>New York</td></tr><tr><td>Bob</td><td>30</td><td>Los Angeles</td></tr><tr><td>Charlie</td><td>35</td><td>Chicago</td></tr></table>', 'languages': ['eng'], 'filetype': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet', 'category': 'Table', 'element_id': '5500e7944e954d7d7aedbf5eb129602d'}


### 文档切割

In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# 将长文本拆分成更小块（chunks）
# 通过递归的方式拆分文本，使得每个文本块的大小符合设定的限制

with open("./test_load/example.txt") as f:
    example_doc = f.read()
    
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 20, # 每个拆分后的文本块的最大长度（以字符为单位）
    chunk_overlap  = 10, # 相邻两个文本块之间的重叠部分的长度（以字符为单位）
    length_function = len, # 计算文本长度的函数
)

texts = text_splitter.create_documents([example_doc])
for text in texts:
    print(text, '\n')

page_content='This is an example' 

page_content='example of the txt' 

page_content='the txt format.' 

page_content='You are all on your' 

page_content='on your own.' 



### 用户输入的文件类型是各种各样的。
但是不同的类型文件需要不同的loader去解析。
现在实现一个方法，不管用户给的是.dm .json .txt .csv .pdf .xlsx 中的哪一种文件，都能读取出来

In [22]:
import os
from langchain_community.document_loaders import TextLoader, JSONLoader, UnstructuredExcelLoader, UnstructuredHTMLLoader, PyPDFLoader
import random


def load_file(file_path):
    # 获取文件类型
    _, file_type = os.path.splitext(file_path)
    file_type = file_type.lower()
    # 不同文件类型传入不同处理函数
    if file_type == ".md" or file_type == ".txt" or file_type == ".csv":
        loader = TextLoader(file_path)
    elif file_type == ".json":
        loader = JSONLoader(file_path=file_path,jq_schema=".test")
    elif file_type == ".xlsx":
        loader = UnstructuredExcelLoader(file_path, mode="elements")
    elif file_type == ".pdf":
        loader = PyPDFLoader(file_path)
    elif file_type == ".html":
        loader = UnstructuredHTMLLoader(file_path)
    else:
        print('Unknown file type')
    docs = loader.load()
    print(docs[0].page_content) 
    print(docs[0].metadata)

# 测试
type_list = ['.md', '.txt', '.csv', '.json', '.xlsx', '.pdf', '.html']
random_choice = random.choice(type_list)
print('测试用例', random_choice, '类型')
random_type_file="./test_load/example"+random_choice
load_file(random_type_file)


测试用例 .csv 类型
Name,Age,City
Alice,25,New York
Bob,30,Los Angeles
Charlie,35,Chicago

{'source': './test_load/example.csv'}
